In [ ]:
import os
import requests
import pandas as pd
from typing import Dict, List
from sqlalchemy import create_engine
from dotenv import load_dotenv
import re
from research.processing.utils import fetch_last_news
from multiprocessing import Pool
from tqdm.autonotebook import tqdm

/tmp/ipykernel_76104/3318184949.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
load_dotenv()
PG_DSN = (
    f"postgresql://{os.getenv('PG_USER')}:"
    f"{os.getenv('PG_PASS')}@"
    f"{os.getenv('PG_HOST')}:"
    f"{os.getenv('PG_PORT')}/"
    f"{os.getenv('PG_DB')}"
)
OPENROUTER_KEY = os.getenv('OPENROUTER_KEY')
engine = create_engine(PG_DSN)

In [ ]:
EXPECTED_KEYS = {"easy", "medium", "hard"}

def parse_gpt_output(raw: str, *, article_id: str = "") -> Dict[str, str]:
    out = {k: "" for k in EXPECTED_KEYS}

    for line in raw.splitlines():
        if not line.strip():
            continue

        m = re.match(r"^\s*([\w]+)\s*[:\-–—]\s*(.+)$", line)
        if not m:
            continue

        k, v = m.group(1).lower(), m.group(2).strip()
        if k in EXPECTED_KEYS:
            out[k] = v

    return out


In [5]:
class GPTSynthesizer:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.url = "https://openrouter.ai/api/v1/chat/completions"

    def _gpt_query(self, system_prompt: str, prompt: str) -> str:
        messages: List[Dict[str, str]] = []
        if system_prompt:
            messages.append({
                "role": "system",
                "content": "Всегда отвечай на русском языке. " + system_prompt
            })
        messages.append({"role": "user", "content": prompt})

        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }

        payload = {
            "model": "qwen/qwen-turbo",
            "messages": messages,
            "temperature": 0,
            "max_tokens": 512,
            "seed": 42,
        }

        response = requests.post(self.url, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]

    def generate_for_article(self, title: str, anons: str, body: str, *, article_id: str = "") -> Dict[str, str]:
        """
        Для переданной новости (title, anons, body) формирует 3 поисковых запроса
        с разными уровнями сложности: лёгкий, средний, сложный.
        Возвращает словарь {"easy": ..., "medium": ..., "hard": ...}.
        """
        snippet = body[:200].replace("\n", " ")
        full_context = f"Заголовок: {title}\nАнонс: {anons or ''}\nТело (фрагмент): {snippet}..."
        system_prompt = (
            "Ты генерируешь три реалистичных поисковых запроса человека в поисковике разного уровня сложности, "
            "чтобы найти именно эту новость на новостном портале. "
            "Первый запрос — максимально буквальный, в точности, как в новости"
            "Второй — чуть более обобщённый (с синонимами). Как бы мог сформулировать эту новость человек"
            "Третий — максимально абстрактный, где используются контекстные формулировки и косвенные признаки, "
            "возможны реалестичные опечатки, чтобы тестировать поисковую систему по новостям. Главное, чтобы запросы были, как от реального человека"
        )
        prompt = (
            f"{full_context}\n\n"
            "Сгенерируй три отдельных строки запроса человека к новостному порталу, помеченные как:\n"
            "easy: \n"
            "medium: \n"
            "hard: "
        )

        raw = self._gpt_query(system_prompt, prompt)

        parsed = parse_gpt_output(raw, article_id=article_id)
        return parsed

In [ ]:
df_sample = fetch_last_news(limit=10000)
gpt = GPTSynthesizer(api_key=OPENROUTER_KEY)

In [ ]:
def _process_row_tuple(args):
    art_id, title, anons, body = args
    try:
        gens = gpt.generate_for_article(title, anons, body, article_id=art_id)

    except Exception as e:
        print(f"[Error] article_id={art_id} → {type(e).__name__}: {e}")
        gens = {"easy": "", "medium": "", "hard": ""}
    return art_id, gens


def generate_synthetic_mp(df, num_workers=None, chunksize=10):
    arg_list = [
        (row["id"], row["title"], row["anons"], row["body"])
        for _, row in df.iterrows()
    ]

    synthetic = []
    with Pool(processes=num_workers) as pool:
        for art_id, gens in tqdm(
            pool.imap_unordered(_process_row_tuple, arg_list, chunksize=chunksize),
            total=len(arg_list),
            desc="Generating synthetic queries",
        ):
            synthetic.append((art_id, "easy",   gens.get("easy", "")))
            synthetic.append((art_id, "medium", gens.get("medium", "")))
            synthetic.append((art_id, "hard",   gens.get("hard", "")))

    return synthetic

In [16]:
synthetic_queries = generate_synthetic_mp(df_sample, num_workers=None, chunksize=5)

Generating synthetic queries: 100%|██████████| 10000/10000 [1:12:12<00:00,  2.31it/s]


In [17]:
df_synth = pd.DataFrame(synthetic_queries, columns=["article_id", "difficulty", "query"])
queries_gt: Dict[str, List[str]] = {}
for art_id, difficulty, q in tqdm(synthetic_queries):
    queries_gt[q] = [art_id]

100%|██████████| 30000/30000 [00:00<00:00, 246646.39it/s]


In [18]:
df_synth.to_csv("synthetic_queries_first_10000.csv", index=False)